In [9]:
% matplotlib inline
from scipy.stats import invgamma
from scipy.stats import norm
from multiprocessing import Pool
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [27]:
def integral(estimate, ts):
    elements = np.ones(len(ts) - 1)
    for i in range(len(ts) - 1):
        elements[i] = (ts[i+1] - ts[i])*(estimate[i+1] + estimate[i])/2
    return sum(elements)

def sum1(beta):
    return np.sum(p[:, 0] - beta * p[:, 3])

def sum2(alpha):
    return np.sum(p[:, 3] * (p[:, 0] - alpha))

def sum3(alpha, beta):
    l = (p[:, 0] - alpha - beta*p[:, 3])
    return np.sum(l * l )

def loglike(alpha, beta, sigma):
    return N*np.log(1/(np.sqrt(2*np.pi*sigma))) - sum3(alpha, beta) / (2*sigma)

def function1(u):
    
    if u < m:
        r = np.random.RandomState(u)

        for i in range(n+1):
            t = ts[i]
            
            if i == 0:
                alphas[0] = 3000
                betas[0] = 185
                sigmas[0] = 90000

            else:
                alphas[0] = np.mean(alpha_sample)
                betas[0] = np.mean(beta_sample)
                sigmas[0] = np.mean(sigma_sample)

            for j in range(sample_iter -1):

                location_alpha = (sigma_alpha*t*sum1(betas[j]) + sigmas[j]*mu_alpha) / (sigma_alpha * N*t + sigmas[j])
                scale_alpha = np.sqrt((sigma_alpha * sigmas[j]) / (sigma_alpha * N*t + sigmas[j]))
                alphas[j+1] = r.normal(loc = location_alpha, scale = scale_alpha)

                location_beta = (sigma_beta * t * sum2(alphas[j+1]) + sigmas[j] * mu_beta) / (sigma_beta *t* ssx + sigmas[j])
                scale_beta = np.sqrt((sigmas[j] * sigma_beta) / (sigma_beta *t* ssx + sigmas[j]))
                betas[j+1] = r.normal(loc = location_beta, scale = scale_beta)

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum3(alphas[j+1], betas[j+1]) + 2)
                sigmas[j+1] = invgamma.rvs(a = shape, scale = 1/ invrate, random_state = r)

            alpha_sample = alphas[burn_in:]
            beta_sample = betas[burn_in:len(betas)]
            sigma_sample = sigmas[burn_in:len(sigmas)]

            box = np.ones(len(alpha_sample))
            for k, l in enumerate(alpha_sample):
                box[k] = loglike(l, beta_sample[k], sigma_sample[k])
                
            estimates[i] = np.average(box)
    
    print(estimates)
    return estimates
    
def sum4(beta):
    return np.sum(p[:, 0] - beta * p[:, 4])

def sum5(alpha):
    return np.sum(p[:, 4] * (p[:, 0] - alpha))

def sum6(alpha, beta):
    l = (p[:, 0] - alpha - beta*p[:, 4])
    return np.sum(l * l )  

def loglike2(alpha, beta, sigma):
    return N*np.log(1/(np.sqrt(2*np.pi*sigma))) - sum6(alpha, beta) / (2*sigma)
    
def function2(w):
    if w < m:
        r = np.random.RandomState(w)

        for i in range(n+1):
            t = ts[i]
            
            if i == 0:
                gammas[0] = 3000
                deltas[0] = 185
                taus[0] = 90000

            else:
                gammas[0] = np.mean(gamma_sample)
                deltas[0] = np.mean(delta_sample)
                taus[0] = np.mean(tau_sample)

            for j in range(sample_iter - 1):

                location_alpha = (sigma_alpha*t*sum4(deltas[j]) + taus[j]*mu_alpha) / (sigma_alpha * N*t + taus[j])
                scale_alpha = np.sqrt((sigma_alpha * taus[j]) / (sigma_alpha * N*t + taus[j]))
                gammas[j+1] = r.normal(loc = location_alpha, scale = scale_alpha)

                location_beta = (sigma_beta * t * sum5(gammas[j+1]) + taus[j] * mu_beta) / (sigma_beta *t* ssz + taus[j])
                scale_beta = np.sqrt((taus[j] * sigma_beta) / (sigma_beta *t* ssz + taus[j]))
                deltas[j+1] = r.normal(loc = location_beta, scale = scale_beta)

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum6(gammas[j+1], deltas[j+1]) + 2)
                taus[j+1] = invgamma.rvs(a = shape, scale = 1/ invrate, random_state = r)

            gamma_sample = gammas[burn_in:]
            delta_sample = deltas[burn_in:len(deltas)]
            tau_sample = taus[burn_in:len(taus)]

            box2 = np.ones(len(gamma_sample))
            for k, l in enumerate(gamma_sample):
                box2[k] = loglike2(l, delta_sample[k], tau_sample[k])

            estimates[i] = np.average(box2)
        
    return estimates

In [29]:
pine = pd.read_table("pine.txt", delim_whitespace = True)
p = pine.values
pine['ave_x'] = pine['x'] - np.average(p[:, 1])
pine['ave_z'] = pine['z'] - np.average(p[:, 2])
p = pine.values

mu_alpha = 3000
sigma_alpha = 10**6
mu_beta = 185
sigma_beta = 10**4
a = 3
b = 1/(2*300**2)

N = np.shape(p)[0]
ssx = np.sum(p[:, 3] * p[:, 3])
ssz = np.sum(p[:, 4] * p[:, 4])

n = 10
c = 2
ts = [(i/n)**c for i in range(n+1)]
estimates = np.ones(n+1)

####ここをいじってね。
sample_iter = 1000
burn_in = 30
m = 100
####

alphas = np.ones(sample_iter)
betas = np.ones(sample_iter)
sigmas = np.ones(sample_iter)

gammas = np.ones(sample_iter)
deltas = np.ones(sample_iter)
taus = np.ones(sample_iter)

# computing BF


model1 marginal likelihood by power posterior

In [30]:
result1 = Pool().map(function1, range(m))

[-748.61052864 -371.98997987 -327.1178053  -316.30146286 -311.03473244
 -308.76030241 -307.05088867 -306.10729292 -305.51266559 -305.08946462
 -304.70187879]
[-683.1822422  -372.0624744  -327.67031172 -315.86350085 -311.04499678
 -308.57135623 -307.23815994 -306.11069065 -305.56819714 -305.04013095
 -304.70037153]
[-757.50620989 -371.24808085 -327.21328954 -316.33122771 -310.96334787
 -308.48525039 -307.13394713 -306.15452103 -305.5481172  -305.07154749
 -304.80352469]
[-729.45015755 -373.44268874 -328.58393626 -316.04721418 -311.01368104
 -308.47129088 -307.06599825 -306.22890735 -305.46582641 -305.17638635
 -304.68451011]
[-718.87726802 -371.70591737 -327.66409963 -316.55080316 -311.38704158
 -308.8605855  -307.08163036 -305.95211016 -305.43131602 -305.05619238
 -304.72329261]
[-695.33823917 -370.12493288 -328.26221541 -316.502299   -311.41553437
 -308.6985797  -307.17683175 -306.30102149 -305.5004337  -305.03059833
 -304.75798911]
[-733.67219857 -376.3496885  -327.09757627 -316.7265

In [26]:
result1

[array([-710.81623026, -370.72662002, -328.64042008, -316.21778264,
        -311.50331799, -308.75339189, -307.11191967, -306.17151279,
        -305.49030224, -304.99074442, -304.77871922]),
 array([-710.81623026, -370.72662002, -328.64042008, -316.21778264,
        -311.50331799, -308.75339189, -307.11191967, -306.17151279,
        -305.49030224, -304.99074442, -304.77871922]),
 array([-710.81623026, -370.72662002, -328.64042008, -316.21778264,
        -311.50331799, -308.75339189, -307.11191967, -306.17151279,
        -305.49030224, -304.99074442, -304.77871922]),
 array([-710.81623026, -370.72662002, -328.64042008, -316.21778264,
        -311.50331799, -308.75339189, -307.11191967, -306.17151279,
        -305.49030224, -304.99074442, -304.77871922]),
 array([-710.81623026, -370.72662002, -328.64042008, -316.21778264,
        -311.50331799, -308.75339189, -307.11191967, -306.17151279,
        -305.49030224, -304.99074442, -304.77871922]),
 array([-710.81623026, -370.72662002, -328.64

In [14]:
expect1 = np.ones(10)
for i in range(10):
    expect1[i] = integral(result1[i], ts)

In [15]:
expect1

array([-311.12526488, -311.263866  , -311.50583205, -311.30598936,
       -311.12828261, -311.52443833, -311.33110477, -311.48110953,
       -311.52057671, -311.36185032])

model2 marginal likelihood by power posterior

In [16]:
result2 = Pool().map(function2, range(m))

In [17]:
result2

[array([-674.69827923, -363.58731507, -319.23566903, -307.38039769,
        -302.56250226, -300.08441022, -298.72689154, -297.57703059,
        -297.06319305, -296.51749186, -296.17703802]),
 array([-687.16164872, -361.80931395, -320.02927451, -308.19658969,
        -302.96242482, -300.26203278, -298.67188541, -297.78462988,
        -296.97936519, -296.49591046, -296.22713534]),
 array([-733.29887937, -361.95142827, -319.17635897, -308.04096642,
        -302.90473819, -300.40217785, -298.74018469, -297.71932227,
        -297.04302104, -296.5357745 , -296.24692049]),
 array([-694.08435972, -363.57697916, -320.1518423 , -307.80486715,
        -302.77646885, -300.3856335 , -298.51659472, -297.62170028,
        -296.97860027, -296.59612894, -296.22967932]),
 array([-653.38290857, -364.98184872, -320.596728  , -307.47506493,
        -302.92916499, -300.01489302, -298.69346617, -297.52163981,
        -297.03849854, -296.58263717, -296.27306047]),
 array([-742.52101025, -365.32447072, -319.56

In [17]:
expect2 = np.ones(10)
for i in range(10):
    expect2[i] = integral(result2[i], ts)

In [18]:
expect2

array([-302.61901882, -302.78617275, -303.00393965, -302.81173615,
       -302.62796775, -303.06430497, -302.82589461, -302.98016589,
       -303.02462573, -302.87313549])

BF21

In [19]:
bf_21 = []
for a,b in zip(expect1, expect2):
    bf_21.append(np.exp(b-a))

In [20]:
bf_21

[4945.5628128686149,
 4806.350060063829,
 4924.0783315006802,
 4886.6056925781786,
 4916.3165350658282,
 4722.6877612537937,
 4940.4424232527035,
 4919.4088039358894,
 4894.9090686738127,
 4859.6166477149563]